In [4]:
import math 
import cmath
import numpy as np
np.set_printoptions(precision=4,suppress=True)

provider = None

def bloch_v(euler_angle):
    theta,phi = euler_angle
    s = math.cos(theta/2)
    t = cmath.exp(phi*1j)*math.sin(theta/2)
    return np.array([s,t])


def bloch_bloch(euler_angle):
    theta,phi = euler_angle
    s = math.cos(theta)
    t = cmath.exp(phi*1j)*math.sin(theta)
    return np.array([t.real,t.imag,s])

bloch_bloch_ans = {
    'zero': (0,0,1),
    'one': (0,0,-1),
    'plus': (1,0,0),
    'minus': (-1,0,0),
    'left': (0,1,0),
    'right': (0,-1,0)
}

bloch_v_ans = {
    'zero': (1,0),
    'one': (0,1),
    'plus': np.array([1,1])/math.sqrt(2),
    'minus': np.array([1,-1])/math.sqrt(2),
    'left': np.array([1,1j])/math.sqrt(2),
    'right': np.array([1,-1j])/math.sqrt(2)  
}

def exercise_A(eas):
    correct, incorrect = 0, 0
    for ea in eas:
        ea_np = np.array(eas[ea])
        
        b_b = bloch_bloch(ea_np)
        if np.allclose(b_b,bloch_bloch_ans[ea]):
            correct += 1
        else:
            incorrect += 1
            
        b_v = bloch_v(ea_np)
        if np.allclose(b_v,bloch_v_ans[ea]):
            correct += 1
        else:
            incorrect += 1
        print(f'{ea}:\n\tangles: {ea_np}\n\t(x,y,z): {b_b}\n\tvector: {b_v}')
    return (correct, incorrect)



# ===============================================s
#
# Find the correct Euler Angles for +, -, left and right
#

euler_angles = {
    'zero': (0,0),
    'one' : (math.pi,0),
    'plus': (math.pi/2,0),       # first entry: theta, second: global phase
    'minus' : (-math.pi/2,0),     
    'left':   (math.pi/2,math.pi/2),     
    'right':  (-math.pi/2,math.pi/2),     
}

result = exercise_A(euler_angles)
if result[0]==12:
    print(f'PASSED')
else:
    print(f'\nDid not pass, {result[0]} correct, {result[1]} incorrect')

zero:
	angles: [0 0]
	(x,y,z): [0. 0. 1.]
	vector: [1.+0.j 0.+0.j]
one:
	angles: [3.1416 0.    ]
	(x,y,z): [ 0.  0. -1.]
	vector: [0.+0.j 1.+0.j]
plus:
	angles: [1.5708 0.    ]
	(x,y,z): [1. 0. 0.]
	vector: [0.7071+0.j 0.7071+0.j]
minus:
	angles: [-1.5708  0.    ]
	(x,y,z): [-1.  0.  0.]
	vector: [ 0.7071+0.j -0.7071+0.j]
left:
	angles: [1.5708 1.5708]
	(x,y,z): [0. 1. 0.]
	vector: [0.7071+0.j     0.    +0.7071j]
right:
	angles: [-1.5708  1.5708]
	(x,y,z): [-0. -1.  0.]
	vector: [ 0.7071+0.j     -0.    -0.7071j]
PASSED


In [65]:
import numpy as np
np.set_printoptions(precision=2,floatmode='fixed',suppress=True)

def make_projector(e):
    s = np.vdot(e,e)
    f = e.conjugate().reshape((2,1))
    return e*f/s

def make_observable(eigenvalue1,eigenvector1,eigenvalue2,eigenvector2):
    e1 = make_projector(eigenvector1)
    e2 = make_projector(eigenvector2)
    return eigenvalue1*e1 + eigenvalue2*e2


obs_ans = {
    'Z': np.array([1.0,0.0,0.0,-1.0]).reshape((2,2)),
    'X': np.array([0.0,1.0,1.0,0.0]).reshape((2,2)),
    'Y': np.array([0.0,1.0j,-1.0j,0.0]).reshape((2,2)),
}


def exercise_B(obss):
    correct, incorrect = 0,0
    for obs in obss:
        e_val1, e_vec1, e_val2, e_vec2 = obss[obs]
        e_vec1 = np.array(e_vec1)
        e_vec2 = np.array(e_vec2)
        m = make_observable(e_val1,e_vec1,e_val2,e_vec2)
        print(f'observation {obs}:\n{m}')
        if np.allclose(m,obs_ans[obs]):
            correct += 1
        else:
            incorrect +=1
    return(correct,incorrect)
 

# ===============================================s
#
# Find the correct eigenvectors for X and Y
#

observables = {
    'Z': (1.0,[1.0,0.0],-1.0,[0.0,1.0]),
    'X': (1.0,[1.0,1.0],-1.0,[1.0,-1.0]),    # |-> and |+> states
    'Y': (1.0,[1.0,1.0j],-1.0,[1.0,-1.0j]), 
}


result = exercise_B(observables)
if result[0]==3:
    print(f'PASSED!')
else:
    print(f'did not pass.')
    print(f'correct {result[0]}, incorrect: {result[1]}')

observation Z:
[[ 1.00  0.00]
 [ 0.00 -1.00]]
observation X:
[[0.00 1.00]
 [1.00 0.00]]
observation Y:
[[0.00+0.00j 0.00+1.00j]
 [0.00-1.00j 0.00+0.00j]]
PASSED!


In [70]:
Z = np.array([1,0,0,-1]).reshape((2,2))
Y = np.array([0,-1j,1j,0]).reshape((2,2))


def test_H_tilde(HT):
    ht_t = np.conj(HT.T)
    y = np.matmul(TH,np.matmul(Z,ht_t))
    if np.allclose(y,Y):
        print('PASSED')
        return True
    print('did not pass')
    print('your result:')
    print(y)
    print('required result:')
    print(Y)
    return False


# ==============================================
#
# find the correct matrix TH
#

TH = np.array([1/2,-1/2,1j,1j]).reshape((2,2)) 

test_H_tilde(TH)

PASSED


True